In [ ]:
%reset

In [2]:
%run './Functions.ipynb'

/home/katya/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


# 0. Functions

In [33]:
#Transforming weights into a heatmap-like df

def make_tables(chunk, n):
    index = set([x[-1] for x in unique_mutations[chunk]])
    columns = set([int(x[:-1]) for x in unique_mutations[chunk]])
    df_0 = pd.DataFrame(index=sorted(index), columns=columns).astype(float) # dummy dfs for each of the entrance neurons
    df_1 = pd.DataFrame(index=sorted(index), columns=columns).astype(float)
    
    for i in range(len(train_weights[chunk][n])):
        position = unique_mutations[chunk][i][:-1]
        mutation = unique_mutations[chunk][i][-1]
        df_0[int(position)][mutation] = (train_weights[chunk][n][i][0]).astype(float)
        df_1[int(position)][mutation] = (train_weights[chunk][n][i][1]).astype(float)
        
    return df_0,df_1

In [34]:
def extract_differences(chunk, n):
    mask = {}
    df = {}
    
    mask[0],mask[1] = make_tables(chunk,n)
    for key in mask:
        mask[key][mask[key]>0]=1
        mask[key][mask[key]<0]=-1
    
    mask = (mask[0] - mask[1]).fillna(0)
    df[0],df[1] = make_tables(chunk,n)
    
    return (df[0][mask!=0]-df[1][mask!=0]).fillna(0)

In [35]:
def count_occurences(chunk,n):
    effects = {}
    for n in range(n):
        result = extract_differences(chunk, n)
        for i in result.index:
            for j in result.columns:
                if result[j].ix[i]!=0:
                    if str(j)+str(i) not in effects.keys():
                        effects[str(j)+str(i)] = []
                        effects[str(j)+str(i)].append(result[j].ix[i])
                    else:
                        effects[str(j)+str(i)].append(result[j].ix[i])
    return effects

# 1. Training models and extracting weights

### 1.1. Test conditions on one sample

In [28]:
chunk = 'S5'

data, labels = read_data(chunk)

train_weights[chunk]={}
cv = cross_validation.StratifiedKFold(labels, n_folds = 20, shuffle = True)
counter = 0

for train_idx, test_idx in cv:
    print (counter)
    X_train, y_train = data[train_idx,:], labels[train_idx]
    X_test, y_test = data[test_idx,:], labels[test_idx]

    model = Sequential()
    init = initializers.Orthogonal(gain=1.0, seed=None)

    model.add(Dense(2, input_dim=data.shape[1], activation='sigmoid', kernel_initializer=init))
    model.add(Dense(100, activation='sigmoid', kernel_initializer=init))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=init))

    opt = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)

    model.compile(optimizer=opt,
                  loss='mean_squared_error',
                  verbose=0)

    model.fit(X_train, y_train, epochs=500, batch_size=500, shuffle=True, verbose=0)
    train_weights[chunk][counter] = model.layers[0].get_weights()[0]
    counter += 1

/home/katya/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=20.
  % (min_labels, self.n_folds)), Warning)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [38]:
for chunk in ['S5']:
    print chunk
    for i in range(20):
        _,_,R,_,_ = stats.linregress(train_weights[chunk][i][:,0],train_weights[chunk][i][:,1])

        print chunk,i,R

S5
S5 0 0.240440462253
S5 1 0.250738901426
S5 2 0.2650773999
S5 3 0.277421680114
S5 4 -0.0740884483528
S5 5 0.276924692172
S5 6 0.391093456367
S5 7 -0.395858567387
S5 8 -0.134742180854
S5 9 0.211869312216
S5 10 0.207364638688
S5 11 0.3699454901
S5 12 0.258245815706
S5 13 -0.439798662858
S5 14 -0.130604731684
S5 15 0.0508628012192
S5 16 -0.499467209335
S5 17 0.480540452372
S5 18 -0.489251086085
S5 19 0.372092915496


In [37]:
toprint = {}
for chunk in ['S5']:
    print chunk
    d = count_occurences(chunk,20)
    for key in d:
        if len(d[key])>10:
            print key+'\t'+str(np.median([np.abs(x) for x in d[key]]))+'\t'+str(np.std([np.abs(x) for x in d[key]]))+'\t'+str(len(d[key]))
    print '\n'

S5
2T	0.543092928827	0.165919797132	11
25L	1.61543478072	0.448285557388	14
21A	2.54797014594	1.98931401822	14
25S	0.929264426231	0.253345615892	12
22S	2.54611752182	0.68615643076	11




### 1.2. Performing the operation on all the samples

In [ ]:
train_weights = {}

for chunk in chunks:
    print chunk
    data, labels = read_data(chunk)
    train_weights[chunk]={}
    cv = cross_validation.StratifiedKFold(labels, n_folds = 10, shuffle = True)
    counter = 0

    for train_idx, test_idx in cv:
        print (counter)
        X_train, y_train = data[train_idx,:], labels[train_idx]
        X_test, y_test = data[test_idx,:], labels[test_idx]

        model = Sequential()
        init = initializers.Orthogonal(gain=1.0, seed=None)

        model.add(Dense(2, input_dim=data.shape[1], activation='sigmoid', kernel_initializer=init))
        model.add(Dense(100, activation='sigmoid', kernel_initializer=init))
        model.add(Dense(1, activation='sigmoid', kernel_initializer=init))

        opt = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)

        model.compile(optimizer=opt,
                      loss='mean_squared_error',
                      verbose=0)

        model.fit(X_train, y_train, epochs=500, batch_size=500, shuffle=True, verbose=0)
        train_weights[chunk][counter] = model.layers[0].get_weights()[0]
        counter += 1

S1
0
1
2
3
4
5
6
7
8
9
S2
0
1
2
3
4
5
6
7
8
9
S3
0
1
2
3
4
5
6
7
8
9
S4
0
1
2
3
4
5
6
7
8
9
S5
0
1
2
3
4
5
6
7
8
9
S6
0
1
2
3
4
5
6
7
8
9
S7
0
1
2
3
4
5
6
7
8
9
S8
0
1
2
3
4
5
6
7
8
9
S9
0
1
2
3
4
5
6
7
8
9
S10
0
1
2
3
4
5
6
7
8
9
S11
0
1
2
3
4
5
6
7
8
9
S12
0
1


In [44]:
toprint = {}
for chunk in chunks:
    print chunk
    d = count_occurences(chunk,10)
    for key in d:
        if len(d[key])>5:
            print key+'\t'+str(np.median([np.abs(x) for x in d[key]]))+'\t'+str(np.std([np.abs(x) for x in d[key]]))+'\t'+str(len(d[key]))
    print '\n'

S1
4H	0.256101656705	0.0796167661781	6
4T	1.35458293557	0.220672464825	7
6K	0.423406101763	0.0985651760815	6
4K	0.603922598064	0.10151636553	8
5F	21.1851017475	0.292305774399	6


S2
8K	0.527602992952	0.0806788443517	8
1S	1.34893192351	0.178351086473	8
8N	3.15251153708	0.345085258834	8
21S	1.71511736512	0.207050942125	8
6F	0.604521650821	0.102642865217	6
27I	7.08989095688	1.27785997362	8
27L	3.44658368826	0.465562356864	8
8D	1.91366416216	0.255324048873	8
27V	6.07503533363	0.784572653476	8
1G	3.17052608728	0.319344766277	8
4F	0.704073920846	0.0831450806891	8
3A	1.90419578552	0.249702121418	8


S3
3K	0.535762486979	0.0335709051574	6
2F	0.464816056192	6.1589637276	6


S4
7G	4.15578126907	0.66110265993	7
29L	1.41330373287	0.39365061593	7
27L	1.79469311237	0.292450269436	6
25F	3.58748015761	0.557603057451	6
1S	1.40971648693	0.148727564692	6
1A	1.38487267494	0.137882376844	7
7N	1.69000315666	0.254892351931	7
0A	1.97161889076	0.264563327929	7
7S	1.38491070271	0.175842620734	6
24S	3.5137465000

In [46]:
for chunk in chunks:
    print chunk
    for i in range(10):
        _,_,R,_,_ = stats.linregress(train_weights[chunk][i][:,0],train_weights[chunk][i][:,1])
        if R < 0:
            print chunk,i,R

S1
S1 0 -0.163736130272
S1 1 -0.0848162587423
S1 3 -0.0892629273228
S1 6 -0.158001951655
S1 8 -0.0778007124799
S1 9 -0.115152006921
S2
S2 0 -0.148251423453
S2 1 -0.632572631964
S2 2 -0.199347823847
S2 3 -0.0700274998789
S2 4 -0.434532600941
S2 7 -0.68741006671
S2 8 -0.129540711033
S3
S3 0 -0.497751535971
S3 1 -0.166373057584
S3 2 -0.446516641084
S4
S4 0 -0.250801780835
S4 1 -0.236262701965
S4 2 -0.546811100115
S4 4 -0.524846722615
S4 5 -0.321982429033
S4 8 -0.544406460937
S4 9 -0.00448597045778
S5
S5 1 -0.422746332091
S5 2 -0.446773048918
S5 4 -0.142853910779
S5 5 -0.392121043054
S5 7 -0.455288478739
S5 9 -0.134892193269
S6
S6 0 -0.00244521056348
S6 1 -0.219355472252
S6 2 -0.690484538376
S6 3 -0.233942763307
S6 7 -0.227695207593
S6 8 -0.0347667272776
S6 9 -0.2932886062
S7
S7 1 -0.477956698841
S7 3 -0.061663976539
S7 5 -0.723537423977
S7 7 -0.476020662637
S7 8 -0.557422604667
S7 9 -0.434928798721
S8
S8 5 -0.291010428506
S8 6 -0.68586062369
S9
S9 0 -0.561258219804
S9 1 -0.376456910168
S9